In [1]:
from utils import Score
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

In [2]:
model_ckpt = "/data/davide/models/t5/rankt5/flan-t5-xl-difference-total-30-ga-32/checkpoint-1/"
score_strategy = "difference"

get_score = getattr(Score, score_strategy)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
query = "What is the significance of the Turing Award in the field of computer science?"

documents = [
    "Title: Major Milestones in Computer Science. Content: Another significant recognition in the world of computing is the Turing Award. Named after Alan Turing, the father of theoretical computer science and artificial intelligence, the award is often dubbed the 'Nobel Prize of Computer Science.' Established in 1966, it is conferred annually by the Association for Computing Machinery (ACM) to an individual or a group who made major contributions to the field of computer science.",
    "Title: Alan Turing, a Brief Biography. Content: Despite his monumental contributions, Turing faced persecution in his lifetime due to his sexuality. However, his legacy lives on in various ways, one of which is the Turing Award. While this award bears his name and is given in the domain of computer science, its significance in terms of academic recognition remains underappreciated.",
    "Title: Notbale Awards in Various Disciplines. Content: Turning to the realm of computer science, one cannot forget the Turing Award. The Turing Award has been around for decades and is a prestigious accolade, honoring those who have pushed the boundaries of computer science research and innovation. This award is more than just a title; it carries with it a rich history and a commitment to recognizing excellence in computing.",
]

inputs = [
    f"Query: {query} Document: {document} Relevant: " 
    for document in documents
]

features = tokenizer(
    inputs, truncation = True, 
    return_tensors = "pt", max_length = 500, 
    padding = True,
)

input_ids = features.input_ids
attention_mask = features.attention_mask
decode_ids = torch.full((input_ids.size(0), 1),
                         model.config.decoder_start_token_id,
                         dtype=torch.long)

In [6]:
with torch.no_grad():
    output = model(
            input_ids = input_ids.to(device), 
            attention_mask = attention_mask.to(device), 
            decoder_input_ids = decode_ids.to(device),    
        ) 
    logits = output.logits[:,0,:]

scores = get_score(logits).cpu().tolist()